In [50]:
import numpy as np

from mpi4py import MPI

import ufl
from basix.ufl import element, mixed_element

import dolfinx
from dolfinx.mesh import CellType, create_unit_square, locate_entities_boundary

from dolfinx import plot
from dolfinx.fem import (
    Constant,
    Function,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
)
from dolfinx.fem.petsc import LinearProblem



In [56]:
msh = create_unit_square(MPI.COMM_WORLD, 96, 96, CellType.triangle)

P1 = element("Lagrange", msh.basix_cell(), degree=1)
E = mixed_element([P1, P1, P1])
V = functionspace(msh, E)

# U = Function(V)

# u, lamda, f = ufl.TrialFunctions(V)
# u, lamda, f = ufl.TrialFunctions(V)
v = ufl.TrialFunction(V)
u = ufl.TrialFunction(V)
U = Function(V)

u, lamda, f = ufl.TrialFunctions(V)
du, dlamda, df = ufl.TestFunctions(V)

alpha = 1.0e-4
zero = Constant(msh, (0.0))

z = Function(functionspace(msh, ("Lagrange", 3)))
z.interpolate(lambda x: x[0]*(1.0 - x[0])*x[1]*(1.0 - x[1]))

a1 = ufl.inner(ufl.grad(u), ufl.grad(du)) * ufl.dx - ufl.inner(f, du) * ufl.dx
a2 = ufl.inner(ufl.grad(lamda), ufl.grad(dlamda)) * \
    ufl.dx - ufl.inner(u, dlamda) * ufl.dx
a3 = ufl.inner(ufl.grad(lamda), ufl.grad(df)) * \
    ufl.dx + alpha * ufl.inner(f, df) * ufl.dx

L1 = zero * du * ufl.dx
L2 = z * du * ufl.dx
L3 = zero * du * ufl.dx

a = a1 + a2 + a3
L = L1 + L2 + L3

facets = locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 2.0),
)

V0, V1 = V.sub(0), V.sub(1)
dofs0 = locate_dofs_topological(V=V0, entity_dim=1, entities=facets)
bc0 = dirichletbc(value=np.float64(0), dofs=dofs0, V=V0)
dofs1 = locate_dofs_topological(V=V1, entity_dim=1, entities=facets)
bc1 = dirichletbc(value=np.float64(0), dofs=dofs1, V=V1)

bcs = [bc0, bc1]


problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()


# z = Function(functionspace(msh, ("Lagrange", 2)))


# alpha = 1.0e-4
# J = (1.0 / 2.0) * (u - z)**2 * ufl.dx + (alpha /2.0 ) * f**2 * ufl.dx

# # Constraint (Poisson eqn) in weak form (lambda*G). Integration by parts
# # has been applied
# G = -ufl.dot(ufl.grad(u), ufl.grad(lamda)) * ufl.dx + lamda * f * ufl.dx

# # Unconstrained functional (want to find stationary points of I)
# I = J + G

# facets = locate_entities_boundary(
#     msh,
#     dim=(msh.topology.dim - 1),
#     marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 2.0),
# )

# # W0 = W.sub(0)
# # Q, _ = W0.collapse()

# V0, V1 = V.sub(0), V.sub(1)
# dofs = locate_dofs_topological(V=V0, entity_dim=1, entities=facets)
# bc0 = dirichletbc(value=np.float64(0), dofs=dofs, V=V0)
# dofs = locate_dofs_topological(V=V1, entity_dim=1, entities=facets)
# bc1 = dirichletbc(value=np.float64(0), dofs=dofs, V=V1)

# # Compute directional derivative about U in the direction of V
# L =  ufl.derivative(I, U, V)
# # a = -ufl.derivative(L, U) # This is a trick to get the bilinear form


# print(alpha)

In [58]:
_uh = uh.sub(0).collapse()

try:
    import pyvista

    cells, types, x = plot.vtk_mesh(_uh.function_space)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = _uh.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    warped = grid.warp_by_scalar()
    plotter.add_mesh(warped)
    if pyvista.OFF_SCREEN:
        pyvista.start_xvfb(wait=0.1)
        plotter.screenshot("uh_poisson.png")
    else:
        plotter.show()
except ModuleNotFoundError:
    print("'pyvista' is required to visualise the solution.")
    print("To install pyvista with pip: 'python3 -m pip install pyvista'.")


ERROR:root:bad X server connection. DISPLAY=
2025-02-18 12:57:32.639 ( 119.042s) [    FFFF93A92A60]vtkXOpenGLRenderWindow.:456    ERR| vtkXOpenGLRenderWindow (0xaaaab39f5a80): bad X server connection. DISPLAY=


: 